In [4]:
import numpy as np
import xarray as xr

In [5]:
# package imports
import pandas as pd
import numpy as np
import xarray as xr
from glob import glob
import os
import fnmatch
import dateutil
import cv2
from tqdm import tqdm
from datetime import datetime
import time
import argparse

# glob for subdirectories
def find(pattern, path):
    result = [] # initialize the list as empty
    for root, dirs, files in os.walk(path): # walk though the path directory, and files
        for name in files:  # walk to the file in the directory
            if fnmatch.fnmatch(name,pattern):  # if the file matches the filetype append to list
                result.append(os.path.join(root,name))
    return result # return full list of file of a given type

In [ ]:
def ephys_to_dataset(path, dates):
    # path should be something like T:/freely_moving_ephys/ephys_recordings/
    # dates should be a list of dates to include, as str, in the format 010121 for Jan 1st 2021
    
    # get all of the npy files in a list
    ephys_filepaths = []
    for day in dates:
        day_npys = find('*ephys_props.npy',os.path.join(path, day))
        for i in day_npys:
            ephys_filepaths.append(i)
    
    # read in the npys, get metadata, and append into dataset
    for file in ephys_filepaths:
        ephys = np.load(file, allow_pickle=True)
#         base_name = 
        ephys.item().get()
        
#         date = split_base_name[0]; mouse = split_base_name[1]; exp = split_base_name[2]; rig = split_base_name[3]
#         try:
#             stim = '_'.join(split_base_name[4:])
#         except:
#             stim = split_base_name[4:]

In [2]:
path = r'C:\Users\Niell Lab\Desktop'
dates = ['012021']

In [6]:
ephys_filepaths = []
for day in dates:
    day_npys = find('*ephys_props.npy',os.path.join(path, day))
    for i in day_npys:
        ephys_filepaths.append(i)

In [7]:
ephys_filepaths

['C:\\Users\\Niell Lab\\Desktop\\012021\\012021_G6H30P3LN_control_Rig2_hf1_wn_ephys_props.npy',
 'C:\\Users\\Niell Lab\\Desktop\\012021\\012021_G6H30P3LN_control_Rig2_hf2_sparsenoise_ephys_props.npy']

In [117]:
# read in the npys, get metadata, and append into dataset
for file in ephys_filepaths:
    ephys = np.load(file, allow_pickle=True)
    keys = ephys.item().keys()
    for key in keys:
        split_key = key.split('_')
        date = split_key[1]; mouse = split_key[0]; exp = split_key[2]; rig = split_key[3]; unit = split_key[-1]
        for x in [date, mouse, exp, rig, unit]:
            split_key.remove(x)
        stim = '_'.join(split_key)
        unit_data = ephys.item().get(key)
        unit_keys = list(unit_data.keys())
        unit_xr = xr.DataArray([unit_data.get(i) for i in unit_keys], dims=['ephys_params'], coords=[('ephys_params', unit_keys)])
        # add metadata
        unit_xr.attrs['date'] = date; unit_xr.attrs['mouse'] = mouse; unit_xr.attrs['exp'] = exp; unit_xr.attrs['rig'] = rig; unit_xr.attrs['unit'] = unit
        unit_xr.attrs['stim'] = stim; unit_xr.name = key
        if unit == 'unit1':
            all_units_xr = unit_xr.copy()
        else:
            all_units_xr = xr.merge([all_units_xr, unit_xr])

In [118]:
unit_xr

<xarray.DataArray 'G6H30P3LN_012021_control_Rig2_hf2_sparsenoise_unit7' (ephys_params: 7)>
array([array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1]),
       array([[ 0.00048136,  0.00875526,  0.02817776, ...,  0.08713568,
         0.14120389, -0.05152785],
       [ 0.00778924,  0.0559672 ,  0.04127231, ...,  0.10408231,
         0.08851532,  0.05055817],
       [ 0.03504014,  0.05062062,  0.06386708, ...,  0.06496467,
         0.10474519,  0.12831966],
       ...,
       [ 0.09610905,  0.08516642,  0.10444636, ...,  0.1583717 ,
         0.15201563, -0.01091706],
       [-0.01440753,  0.06124212,  0.06431775, ...,  0.03489931,
        -0.02801979, -0.00458239],
       [-0.00829195,  0.09442876,  0.08190549, ..., -0.00587235,
         0.07488788,  0.10940169]]),
       array([       nan,        nan,        nan, 0.        , 0.58651026,
       0.39464412, 0.34134988, 0.47213115, 0.31591738, 0.15748031,
       1.19760479, 0.        ]),
       list([0.0585289523, 0.034482535, 0.0308612846, 0.0211423524, 0.0130883558, 0.0032147982, -0.0072035557, -0.017866889, -0.028351476400000002, -0.0392690115, -0.0510622822, -0.0627294108, -0.07378853860000001, -0.0853066891, -0.0981715992, -0.1110321507, -0.1222509444, -0.1342009008, -0.1510577351, -0.1696101427, -0.1726071835, -0.158229813, -0.1473763138, -0.1399937719, -0.1327670813, -0.1222525612, -0.1114499196, -0.1031980067, -0.09640812130000001, -0.08816444130000001, -0.0787506774, -0.0704646483, -0.0634591654, -0.0562532581, -0.048079382600000005, -0.040367752300000004, -0.0335068405, -0.027030879600000002, -0.0207746364, -0.014701684900000001, -0.0087696128, -0.0034591912, 0.0025346826, 0.0089628343, 0.014827786, 0.019948935100000002, 0.0254853573, 0.030775077600000002, 0.036582663700000004, 0.042287834, 0.0470089018, 0.050888434100000005, 0.054710466400000005, 0.0584943853, 0.061727903800000004, 0.0646912828, 0.0670415387, 0.06740619240000001, 0.067175813, 0.0664719194, 0.0711865872]),
       array([-1.00000000e+00, -9.00000000e-01, -8.00000000e-01, -7.00000000e-01,
       -6.00000000e-01, -5.00000000e-01, -4.00000000e-01, -3.00000000e-01,
       -2.00000000e-01, -1.00000000e-01, -2.22044605e-16,  1.00000000e-01,
        2.00000000e-01,  3.00000000e-01,  4.00000000e-01,  5.00000000e-01,
        6.00000000e-01,  7.00000000e-01,  8.00000000e-01,  9.00000000e-01,
        1.00000000e+00]),
       array([0.1243219 , 0.        , 0.        , 0.        , 0.71479524,
       0.25181013, 0.        , 0.        , 1.01107574, 0.80817263,
       0.39496416, 0.        , 1.12854229, 3.59722832, 2.59371391,
       2.47950611, 3.07961709, 4.74852203, 2.99363665, 7.55650832,
       4.58445654]),
       array([0.46723413, 0.        , 0.        , 0.52777943, 0.62034828,
       1.83095067, 0.59373493, 0.4312669 , 0.26233343, 0.73590238,
       0.        , 0.2868083 , 1.60109505, 0.53749186, 1.55494287,
       1.56620298, 1.74528003, 2.75945234, 2.4473612 , 3.40939109,
       2.45464054])], dtype=object)
Coordinates:
  * ephys_params  (ephys_params) <U17 'contrast_range' 'sta' ... 'downsacc_avg'
Attributes:
    date:     012021
    mouse:    G6H30P3LN
    exp:      control
    rig:      Rig2
    unit:     unit7
    stim:     hf2_sparsenoise

In [96]:
np.array(list(unit_data.items()), dtype=object)

array([['contrast_range',
        array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1])],
       ['sta',
        array([[-0.00028675,  0.00419492,  0.00915174, ...,  0.03980648,
         0.06606979,  0.04277383],
       [ 0.00188483,  0.00983763,  0.01214727, ...,  0.03700616,
         0.04278268,  0.0558139 ],
       [ 0.00451413,  0.00903736,  0.01242749, ...,  0.02364965,
         0.03682209,  0.06296573],
       ...,
       [ 0.02113788,  0.02492579,  0.01448229, ...,  0.03781624,
         0.06371544, -0.02885267],
       [ 0.00506146,  0.02623348,  0.02530186, ..., -0.00176933,
        -0.01261696,  0.00194353],
       [ 0.00036332,  0.02271778,  0.02323743, ..., -0.01324139,
         0.01503561,  0.0240894 ]])],
       ['contrast_response',
        array([        nan, 17.01357466, 17.27666273, 16.18865519, 14.70132239,
       14.15352261, 13.64238411, 12.38208482, 11.29717571, 12.35507246,
       12.09756098, 10.05128205])],
       ['waveform',
        list([0.04436

In [18]:
head, tail = os.path.split(r'C:\Users\Niell Lab\Desktop\012021\012021_G6H30P3LN_control_Rig2_hf1_wn_ephys_props.npy')
base_name = '_'.join(['_'.join(reversed(tail.split('_')[0:2])), '_'.join(tail.split('_')[2:-2])])

In [20]:
base_name

'G6H30P3LN_012021_control_Rig2_hf1_wn'

In [ ]:
ephys_to_dataset(r'C:\Users\Niell Lab\Desktop\', )